In [1]:
import pandas as pd
import numpy as np
import helper_functions as hf
import pickle
import matplotlib.pyplot as plt
import importlib
import math
import copy
import re
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import random
importlib.reload(hf)
import warnings
warnings.filterwarnings('ignore')

data_dir = "C:/Users/ctata/Documents/Lab/quality_vectors_git/data/"
fig_dir = "C:/Users/ctata/Documents/Lab/quality_vectors_git/figures/"

qual_vecs, embed_ids, embed_seqs = hf.getQualVecs(data_dir)



otu = pd.read_csv(data_dir + "halfvarson/seqtab.txt", sep = "\t")
otu.shape

mapping = pd.read_csv(data_dir + "halfvarson/mapping.txt", sep = "\t")
mapping = mapping.set_index("sample_name")


best_hits = pd.read_csv(data_dir + "halfvarson/embed/best_hits.tsv", header = None, sep = "\t")
best_hits.columns = ["query_id", "hit_id", "query_seq", "hit_seq", "evalue", "bitscore"]
best_hits = best_hits.set_index('query_id')
keep = [i < 1E-29 for i in best_hits['evalue'] ]
best_hits = best_hits.loc[keep, :]
print(best_hits.shape)
best_hits


#Get only those ASVs that have a match in the embedding set

best_hits = best_hits.loc[[i in otu.columns.values for i in best_hits['query_seq']], :]
otu_use = otu.loc[:, best_hits['query_seq'].values]

#Assign id of best match 
otu_use.columns = best_hits.loc[:, 'hit_id']

#Put transformation matrix in order to be dotted with the ASV table
qual_vecs_half = qual_vecs.loc[otu_use.columns.values, :]


#Keep only diagnoses we're interested in right now
mapping['sample_name'] = mapping.index.values
keep = [i in ["CD", "UC", "HC"] for i in mapping["diagnosis_full"].values]
mapping = mapping.loc[keep, :]
mapping = mapping.loc[[i in otu_use.index.values for i in mapping.index.values], :]
otu_use = otu_use.loc[mapping.index, :]
otu_all = otu.loc[mapping.index, :]

map_half = mapping
otu_half = otu_use
print(otu_use.shape)
print(otu_all.shape)

26726
26726
(26726, 100)
(25159, 5)
(564, 17775)
(564, 26251)


In [5]:
from collections import Counter
otu_use.shape
mapping.shape
len(Counter(mapping.patientnumber))

118

In [2]:
y_half = [i in ["CD", "UC"] for i in map_half['diagnosis_full']]
embedded_half = pd.DataFrame(np.dot(hf.asinh(otu_half), qual_vecs_half))
embedded_half.columns = qual_vecs_half.columns.values


In [3]:

#Load data
filt = ".07"
f = open(data_dir + "AG_new/filter_.07//otu_train_" + str(filt) + ".obj", "rb")
otu_train_ag = pickle.load(f)
f.close()

f = open(data_dir + "AG_new/filter_.07//otu_test_" + str(filt) + ".obj", "rb")
otu_test_ag = pickle.load(f)
f.close()

f = open("C:/Users/ctata/Documents/Lab/quality_vectors/data/AG_new/feces/map_train_strictDiag.obj", "rb")
map_train_ag = pickle.load(f)
f.close()

f = open("C:/Users/ctata/Documents/Lab/quality_vectors/data/AG_new/feces/map_test_strictDiag.obj", "rb")
map_test_ag = pickle.load(f)
f.close()

qual_vecs_ag, embed_ids, embed_seqs = hf.getQualVecs(data_dir)

otu_train_ag = hf.matchOtuQual(otu_train_ag, qual_vecs_ag.index.values, embed_seqs)
otu_test_ag = hf.matchOtuQual(otu_test_ag, qual_vecs_ag.index.values, embed_seqs)

data_ag = pd.concat([otu_train_ag, otu_test_ag])
mapping_ag = pd.concat([map_train_ag, map_test_ag])
embedded_ag = pd.DataFrame(np.dot(hf.asinh(data_ag), qual_vecs_ag))
embedded_ag.columns = qual_vecs.columns.values
y_ag = mapping_ag["IBD"].values

26726
26726
(26726, 100)
all good
all good


In [4]:
embedded_train_ag = pd.DataFrame(np.dot(hf.asinh(otu_train_ag), qual_vecs_ag))
embedded_test_ag = pd.DataFrame(np.dot(hf.asinh(otu_test_ag), qual_vecs_ag))
y_train_ag = map_train_ag["IBD"].values
y_test_ag = map_test_ag["IBD"].values

In [7]:
importlib.reload(hf)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

data_ag_reord = data_ag.loc[:, otu_half.columns.values]
f = plt.figure(figsize=(15,5))
m, auc, auc_train, fpr, tpr, prec, f1, f2, feat_imp_embed= hf.predictIBD(X_train = hf.asinh(data_ag_reord), y_train = y_ag, X_test = hf.asinh(otu_half), y_test = y_half,
                                                                     max_depth = 5, n_estimators = 170,  weight = 20,
                                                                     plot= False, plot_pr = False, flipped = False)
#f.savefig("../figures/curves_ag_half_asin.pdf")


#probs = m.predict_proba(hf.asinh(otu_half))
preds = m.predict(hf.asinh(otu_half))
#print(preds)
y_flipped = y_half
print("f1", f1)
print("f2" , f2)
print("Accuracy", np.sum( preds == y_flipped) / len(y_flipped) )
print("Precision: ", precision_score(y_flipped, preds))
print("Recall: ", recall_score(y_flipped, preds))
print("0 Precision: ", precision_score([1 - i for i in y_half], 1 - preds))
print("0 Recall: ", recall_score([1 - i for i in y_half], 1 - preds))
print(preds)

f1 0.038461538461538464
f2 0.024390243902439025
Accuracy 0.11347517730496454
Precision:  1.0
Recall:  0.0196078431372549
0 Precision:  0.09747292418772563
0 Recall:  1.0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

<Figure size 1080x360 with 0 Axes>

In [8]:
#whole halfvarson dataset
importlib.reload(hf)
f = plt.figure(figsize=(15,5))
m_embed, auc, auc_train, fpr, tpr, prec, f1, f2, feat_imp_embed = hf.predictIBD(X_train = embedded_ag, y_train = y_ag, X_test = embedded_half, y_test = y_half, 
                                                                            max_depth = 5, n_estimators = 95,  weight = 20,
                                                                            plot= False, plot_pr = False, flipped = False)

preds = m_embed.predict(embedded_half)
y_flipped = y_half

print("f1", f1)
print("f2" , f2)
print("Accuracy", np.sum(preds == y_flipped) / len(y_flipped) )
print("Precision: ", precision_score(y_flipped, preds))
print("Recall: ", recall_score(y_flipped, preds))
print("0 Precision: ", precision_score([1 - i for i in y_flipped], 1 - preds))
print("0 Recall: ", recall_score([1 - i for i in y_flipped], 1 - preds))
print(preds)

f1 0.40873634945397813
f2 0.30170428374021185
Accuracy 0.3280141843971631
Precision:  1.0
Recall:  0.2568627450980392
0 Precision:  0.12471131639722864
0 Recall:  1.0
[1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1
 0 0 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0
 0 0 0 0

<Figure size 1080x360 with 0 Axes>